## Import Dependencies

In [1]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless = False)

In [ ]:
# URL of the NASA Mars News to be scrapped
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(10)

In [ ]:
# Retrieve page with requests module
html = browser.html

In [ ]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')

In [ ]:
# Examine the results, then determine element that contains sought info
soup.prettify()

## Collect the latest News Title and Paragraph Text

In [ ]:
# Get list of all the headlines
headline_lists= soup.find('ul', class_="item_list")

# Collect top headline and title from list of headlines 
latest_headline_title = headline_lists.find('div', class_="content_title").text
latest_headline_para = headline_lists.find('div', class_="article_teaser_body").text

print(f'Latest Headline Title : {latest_headline_title}')
print('****************************************************************************************************')
print(f'Latest Headline Paragraph: {latest_headline_para}' )

## JPL Mars Space Images - Featured Image

In [ ]:
# URL of the NASA Space Image to be scrapped
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
time.sleep(10)
html = browser.html
soup = bs(html, 'html.parser')

In [ ]:
# Extract tag containing url to latest url to current featured mars image
article = soup.find('article', class_="carousel_item")

# Extract url from the article tag
start = article['style'].find("/")
end = article['style'].find("\')")
featured_image_url = "https://www.jpl.nasa.gov" + article['style'][start:end]

print(featured_image_url)

## Mars Weather - Scrape the latest Mars weather tweet from the page

In [ ]:
# URL of Mars weather to be scrapped
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
time.sleep(10)
html = browser.html
soup = bs(html, 'html.parser')
soup.prettify()

In [ ]:
result = soup.find('div', class_="css-901oao r-hkyrab r-1qd0xha r-a023e6 r-16dba41 r-ad9z0x r-bcqeeo r-bnwqim r-qvutc0")
mars_weather = result.find('span', class_="css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0").text.strip()
mars_weather


## Mars Facts

In [ ]:
mars_facts_url = "https://space-facts.com/mars/"
tables = pd.read_html(mars_facts_url)
mars_facts_df = tables[1]

mars_facts_df.head()

In [ ]:
# Convert dataframe to a HTML table string
html_table = mars_facts_df.to_html()
html_table

## Mars Hemispheres

In [4]:
# Initialize browser for navigating dom tree
def initializeBrowser(url):
    browser.visit(url)
    time.sleep(5)
    html = browser.html
    soup = bs(html,'html.parser')
    soup.prettify()
    
    return soup

# Returns dictionary containing hemisphere title and image url
def getMarsHemisphereUrls(soup):
    item_list = soup.find_all('div', class_='item')

    hemisphere_image_url = []
    base_url = "https://astrogeology.usgs.gov"
    
    for item in item_list:
        anchor_tag = item.find('a', class_='itemLink product-item')
        
        image_title = item.find('h3').text.strip().split(' Enhanced')[0]
        url = base_url + anchor_tag['href']
        
        # Navigate to new url to fetch url of high res image
        soup = initializeBrowser(url)
        
        full_res_image_url = soup.find('div', class_="downloads").find_all('li')[1].find('a')['href']
        hemisphere_image_url.append({"title": image_title,"img_url": full_res_image_url})
            
    return hemisphere_image_url


usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
soup = initializeBrowser(usgs_url)

for key in getMarsHemisphereUrls(soup):
    print(key)

{'title': 'Cerberus Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif'}
{'title': 'Schiaparelli Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif'}
{'title': 'Syrtis Major Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif'}
{'title': 'Valles Marineris Hemisphere', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif'}
